In [4]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from geopy.geocoders import Nominatim
import folium
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# enter working directory
#cd = input('enter directory:')

In [5]:
address = 'Quan 7, Ho Chi Minh'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 10.7365728, 106.7224322.


In [6]:
map_dn = folium.Map(location=[latitude, longitude], zoom_start=13)
map_dn

In [28]:
CLIENT_ID = 'ZRHSAYU5TZTMWAMXTJ3HTUEN5CHDJAI1ZXC3STWIPCUVIJZT' # your Foursquare ID
CLIENT_SECRET = 'T2CXAM5OQ2UIINW1BUWDTOAW1W4IXOXB352VTCHJQKWFCA2P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius


request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "Coffee",
    "near": "Quan 1",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [27]:
results = data.json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng,id
0,Starbucks Reserve,Coffee Shop,10.778961,106.698185,59997bb15d0fea4b39463d94
1,Starbucks,Coffee Shop,10.781078,106.700207,554faad7498e38786638336f
2,TCF The Coffee Factory,Coffee Shop,10.780012,106.700768,53cf1253498e7214fa6d24b5
3,Basilico,Italian Restaurant,10.781411,106.701243,4e2b8d117d8b7deda6d2baa4
4,Cộng Cà Phê,Café,10.777917,106.701266,56764e09498eca1773f719b4
...,...,...,...,...,...
79,Hoa Da Coffee,Café,10.784382,106.697087,4d578f80380ab1f7e9e0fdf4
80,Cafe Highland Mạc Đĩnh Chi,Café,10.785363,106.698653,505e7dabe4b011bf1f276520
81,Windows 3 coffee,Café,10.784962,106.696355,4bcef8d4cc8cd13a6318c5cf
82,Trung Nguyen Cafe,Café,10.785349,106.696909,4c7cfc0c278eb713dcba6d80


In [31]:
url1 = 'https://api.foursquare.com/v2/venues/56ab49cc498e877ac0cf32fc?&client_id=ZRHSAYU5TZTMWAMXTJ3HTUEN5CHDJAI1ZXC3STWIPCUVIJZT&client_secret=T2CXAM5OQ2UIINW1BUWDTOAW1W4IXOXB352VTCHJQKWFCA2P&v=20180605'
x = requests.get(url1).json()
x

{'meta': {'code': 200, 'requestId': '5f1bea77fa27bf3620477826'},
 'response': {'venue': {'id': '56ab49cc498e877ac0cf32fc',
   'name': "Pizza 4P's",
   'contact': {},
   'location': {'address': 'Tôn Dật Tiên District 7',
    'lat': 10.731694587962846,
    'lng': 106.71575577202888,
    'labeledLatLngs': [{'label': 'display',
      'lat': 10.731694587962846,
      'lng': 106.71575577202888}],
    'cc': 'VN',
    'city': 'Hồ Chí Minh',
    'country': 'Việt Nam',
    'formattedAddress': ['Tôn Dật Tiên District 7',
     'Hồ Chí Minh',
     'Việt Nam']},
   'canonicalUrl': 'https://foursquare.com/v/pizza-4ps/56ab49cc498e877ac0cf32fc',
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 14},
   'price': {'tier': 1, 'message': 'Cheap

In [120]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1aa3dc1e406e22326a4090'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ho Chi Minh City',
  'headerFullLocation': 'Ho Chi Minh City',
  'headerLocationGranularity': 'city',
  'totalResults': 88,
  'suggestedBounds': {'ne': {'lat': 10.763572827000026,
    'lng': 106.74986199865401},
   'sw': {'lat': 10.709572772999973, 'lng': 106.69500240134599}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56ab49cc498e877ac0cf32fc',
       'name': "Pizza 4P's",
       'location': {'address': 'Tôn Dật Tiên District 7',
        'lat': 10.731694587962846,
        'lng': 106.71575577202888,
        'labeledLatLngs': [{'label': 'display',
          'lat': 10